In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
import boto3
import csv


# get a handle on s3
session = boto3.Session(
                    aws_access_key_id='XXXX',
                    aws_secret_access_key='XXXX',
                    region_name='XXXX')
                    
s3 = session.resource('s3')


# get a handle on the bucket that holds your file
bucket = s3.Bucket('proyectocanopy') # example: energy_market_procesing


# Listing available files
for my_bucket_object in bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_registro_contratistas')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_registro_contratistas.csv')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_registro_regiones')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_registro_regiones.csv')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_valor_contratistas')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_valor_contratistas.csv')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_valor_regiones')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_concentracion_valor_regiones.csv')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_financiacion')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_financiacion.csv')
s3.ObjectSummary(bucket_name='proyectocanopy', key='alarma_transparencia')
s3.Object

In [135]:
import pandas as pd
import s3fs
df = pd.read_csv('s3://proyectocanopy/alarma_transparencia.csv')

In [136]:
df.shape

(22725, 18)

# Transparencia

In [137]:
df_transparency = pd.read_csv('s3://proyectocanopy/alarma_transparencia.csv')
df_transparency.head()

,Unnamed: 0,Unnamed: 0.1,ids,fechas,regiones,departamento,municipio,entidades,contratista_ids,contratista_name,valores,detalles,reporta_parties,reporta_planning,uri,plataforma,reporta_items,departamento_def
0,0,98,20-12-10962842,2020-07-28T15:54:33.000Z,ARAUCA,Arauca,ARAUCA,ARAUCA - UNIDAD ADMINISTRATIVA ESPECIAL DE SAL...,111677609,RUIZ BARCO SOFIA SAHARAY,7189350.0,PRESTACION DE SERVICIOS AUXILIARES COMO APOYO...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,arauca
1,1,140,20-12-10977749,2020-08-03T06:43:56.000Z,NARIÑO,Nariño,BARBACOAS,NARIÑO - BARBACOAS - ALCALDIA,94534533,SILVIO RUA QUIÑONES,1200000.0,BRINDAR APOYO COMO AUXILIAR OPERATIVO DE LA AL...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,nariño
2,2,247,20-4-10903430,2020-07-07T17:21:03.000Z,SANTANDER,NaN,NaN,SANTANDER - COLEGIO INTEGRADO NUESTRA SEÑORA D...,900579939,SOLUCIONES & TECNOLOGIA SAS,9486000.0,"INSUMOS DE FOTOCOPIADORA, Y SUMINISTRO INTEGRA...",SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,NaN
3,3,326,20-12-10903397,2020-07-07T17:13:18.000Z,BOLIVAR,Bolívar,SAN MARTÍN DE LOBA,BOLIVAR - SAN MARTIN DE LOBA - ALCALDIA,23106451,MAVELIS BANDERA BELAIDES,7500000.0,Prestación de servicios de Auxiliar de Enferme...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,bolívar
4,4,339,20-11-10875699,2020-06-30T10:38:07.000Z,BOGOTÁ D.C.,Bogotá D.C.,Bogotá D.C.,BOGOTÁ D.C. - IED. COLEGIO GABRIEL BETANCOURT ...,900682841,EDUCACTIVA SAS,59937000.0,COMPRA DE CARTILLAS GRADOS DE JARDIN A QUI...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,bogotá


In [138]:
# Creating alarm variables
df_transparency['has_alarm'] = (df_transparency['reporta_items']!='SI_REPORTA')
# Filtering
df_transparency = df_transparency[df_transparency['has_alarm']]

# Formatting with $ 
df_transparency['valores'] = pd.to_numeric(df_transparency['valores'], errors='coerce')
df_transparency['valores'] = df_transparency['valores'].map('${:,.0f}'.format)
# Formatting descripcion
df_transparency['detalles'] = df_transparency['detalles'].str.capitalize()
# Formatting fechas with only date
df_transparency['date'] = pd.to_datetime(df_transparency['fechas']).dt.date
# Getting dates' month
df_transparency['month'] =pd.to_datetime(df_transparency['fechas']).dt.to_period("M")
# Capitalizing names of columns
df_transparency['municipio'] = df_transparency['municipio'].str.title()
df_transparency['departamento'] = df_transparency['departamento'].str.title()
df_transparency['contratista_name'] = df_transparency['contratista_name'].str.title()

df_transparency = df_transparency.rename(columns={
    'entidades': 'Nombre de la entidad',
    'departamento': 'Departamento',
    'municipio': 'Municipio',
    'date': 'Fecha del contrato',
    'detalles': 'Descripcion del contrato',
    'contratista_name': 'Proveedor adjudicado',
    'contratista_ids': 'ID Proveedor',
    'uri': 'SECOP URL',
    'valores': 'Valor del contrato',
    'month': 'Mes del contrato'
})

df_transparency_final = df_transparency[[ 'Fecha del contrato', 'Departamento', 'Municipio', 
                                   'Nombre de la entidad', 'Proveedor adjudicado', 
                                    'ID Proveedor', 'Valor del contrato',
                                     'Descripcion del contrato',  'SECOP URL']]
df_transparency_final = df_transparency_final.sort_values(by='Fecha del contrato', ascending=False)

/Users/lmontesmarti/Documents/GitHub/canopy-demo/venv/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1091: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [139]:
df_transparency_final.head()

,Fecha del contrato,Departamento,Municipio,Nombre de la entidad,Proveedor adjudicado,ID Proveedor,Valor del contrato,Descripcion del contrato,SECOP URL
18725,2020-07-05,Tolima,Ibagué,TOLIMA - UNIVERSIDAD DEL TOLIMA,Hamilton Julian Barbosa Vinasco,111052812,"$8,000,000",Apoyo a la gestión para procesar las diferente...,https://www.contratos.gov.co/consultas/detalle...
3351,2020-06-18,NaN,NaN,RISARALDA - INSTITUCION EDUCATIVA RODRIGO AREN...,Luisa Fernanda Marin Zuluaga,100471954,"$11,260,000","Suministrar, dos computadores todo en uno leno...",https://www.contratos.gov.co/consultas/detalle...
16740,2020-05-26,Nariño,Ipiales,NARIÑO - IPIALES - ALCALDIA,Alba Susana Timaran Tibata,108590121,"$11,811,282",Prestación de servicios profesionales de enfer...,https://www.contratos.gov.co/consultas/detalle...
16755,2020-05-26,Antioquia,Girardota,ANTIOQUIA - GIRARDOTA - ALCALDIA,Luis Fernando Castaño Orozco,71646152,"$12,000,000",Prestación de servicios profesionales y de a`p...,https://www.contratos.gov.co/consultas/detalle...
3546,2020-05-26,Bolívar,Arjona,BOLIVAR - ESE HOSPITAL LOCAL ARJONA,Alvaro Alfonso Arrieta Perez,9097905,"$11,638,413",Adición del contrato de obra para la adecuaci...,https://www.contratos.gov.co/consultas/detalle...


In [140]:
print(df_transparency_final['SECOP URL'].isna().sum())
print(len(df_transparency_final))

0
10261


## Graph for transparency

In [141]:
graph_df = df_transparency.groupby(['Mes del contrato']).size().reset_index(name='Num contratos')
# graph_df = graph_df.pivot_table(index='Mes del contrato', columns='Razón alarma', values='Num contracts')
graph_df

,Mes del contrato,Num contratos
0,2020-01,45
1,2020-02,69
2,2020-03,1298
3,2020-04,5708
4,2020-05,3139
5,2020-06,1
6,2020-07,1


In [142]:
fig = go.Figure(data=[
    go.Bar(x=graph_df['Mes del contrato'].astype(str), y=graph_df['Num contratos']),
])
# Change the bar mode
fig.update_layout(
    xaxis_tickformat = '%B <br>%Y'
)

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = graph_df['Mes del contrato'].astype(str).unique(),
        ticktext = graph_df['Mes del contrato'].astype(str).unique()
    )
)
fig.update_layout(
    font=dict(
        color="#252525",
        family="Roboto"
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_title="Fecha del contrato",
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.show()

# 2. Financiación de campañas

In [143]:
financiacion = pd.read_csv('s3://proyectocanopy/alarma_financiacion.csv')
financiacion.head()

,Unnamed: 0,Unnamed: 0.1,ids,fechas,municipio_contrato,entidades,contratista_ids,contratista_name,valores,detalles,...,circunscripcion.electoral,organizacion.politica,identificacion.candidato,nombre.candidato,elegido,valor,tipo.donacion,concepto.del.ingreso,identificacion_aportante,aportante.original.nombre
0,0,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,PARTIDO ALIANZA VERDE - PARTIDO ALIANZA SOCIAL...,71587929,FABIO ALONSO SALAZAR JARAMILLO,No,"$20,000,000",Donación,DONACION PERSONA JURIDICA,890900943,COLOMBIANA DE COMERCIO SA
1,1,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
2,2,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
3,3,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,RECUPEREMOS BOGOTA,19333686,ENRIQUE PEÑALOSA LONDOÑO,Si,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A
4,4,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,CREEMOS CALI,14446558,NORMAN MAURICE ARMITAGE CADAVID,Si,"$10,000,000",Donación,DONACION CHQ 174416 BANCOLOMBIA,890900943,COLOMBIANA DE COMERCIO S.A.


In [144]:
financiacion.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ids', 'fechas', 'municipio_contrato',
       'entidades', 'contratista_ids', 'contratista_name', 'valores',
       'detalles', 'uri', 'plataforma', 'reporta_items', 'departamento_def',
       'corporacion.o.cargo', 'circunscripcion.electoral',
       'organizacion.politica', 'identificacion.candidato', 'nombre.candidato',
       'elegido', 'valor', 'tipo.donacion', 'concepto.del.ingreso',
       'identificacion_aportante', 'aportante.original.nombre'],
      dtype='object')

In [145]:
financiacion[['corporacion.o.cargo', 'circunscripcion.electoral',
       'organizacion.politica', 'identificacion.candidato', 'nombre.candidato',
       'elegido', 'valor', 'tipo.donacion', 'concepto.del.ingreso',
       'identificacion_aportante', 'aportante.original.nombre']]

,corporacion.o.cargo,circunscripcion.electoral,organizacion.politica,identificacion.candidato,nombre.candidato,elegido,valor,tipo.donacion,concepto.del.ingreso,identificacion_aportante,aportante.original.nombre
0,Alcaldía,Municipal,PARTIDO ALIANZA VERDE - PARTIDO ALIANZA SOCIAL...,71587929,FABIO ALONSO SALAZAR JARAMILLO,No,"$20,000,000",Donación,DONACION PERSONA JURIDICA,890900943,COLOMBIANA DE COMERCIO SA
1,Alcaldía,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
2,Alcaldía,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
3,Alcaldía,Municipal,RECUPEREMOS BOGOTA,19333686,ENRIQUE PEÑALOSA LONDOÑO,Si,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A
4,Alcaldía,Municipal,CREEMOS CALI,14446558,NORMAN MAURICE ARMITAGE CADAVID,Si,"$10,000,000",Donación,DONACION CHQ 174416 BANCOLOMBIA,890900943,COLOMBIANA DE COMERCIO S.A.
...,...,...,...,...,...,...,...,...,...,...,...
917,Alcaldía,Municipal,PARTIDO CONSERVADOR COLOMBIANO,8106620,HUGO ANDRES DIAZ VILLA,No,"$5,000,000",Donación,DONACION A LA CAMPAÑA PARA LA ALCALDIA DEL MUN...,800093761,TAX INDIVIDUAL S.A
918,Concejo,Municipal,PARTIDO LIBERAL COLOMBIANO,71310233,ARIES WEST ROWE MATEUS,Si,"$3,000,000",Donación,DONACION PARA GASTOS DE LA CAMPAÑA AL CONCEJO,800093761,EMPRESA TRANSPORTADORA DE TAXIS INDIVIDUAL SA
919,Senado de la República,Nacional Ordinaria,MOVIMIENTO POLITICO TODOS SOMOS COLOMBIA - TCS,71451657,GUILLERMO ENRIQUE GONZALEZ OCHOA,No,"$500,000",Donación,Donación como apoyo a la campaña del candidato...,900270192,CORPORACIÓN VISIÓN GENTE
920,Concejo,Municipal,PARTIDO SOCIAL DE UNIDAD NACIONAL PARTIDO DE LA U,42679912,ADRIANA MARIA JIMENEZ GOMEZ,No,"$1,000,000",Aporte,APORTE CAMPAÑA DE SU PROPIO PATRIMONIO,42679912,ADRIANA MARIA JIMENEZ GOMEZ


In [146]:
financiacion.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ids', 'fechas', 'municipio_contrato',
       'entidades', 'contratista_ids', 'contratista_name', 'valores',
       'detalles', 'uri', 'plataforma', 'reporta_items', 'departamento_def',
       'corporacion.o.cargo', 'circunscripcion.electoral',
       'organizacion.politica', 'identificacion.candidato', 'nombre.candidato',
       'elegido', 'valor', 'tipo.donacion', 'concepto.del.ingreso',
       'identificacion_aportante', 'aportante.original.nombre'],
      dtype='object')

In [147]:
financiacion.shape

(922, 25)

In [148]:
financiacion[financiacion['organizacion.politica'].str.contains('CENTRO')]

,Unnamed: 0,Unnamed: 0.1,ids,fechas,municipio_contrato,entidades,contratista_ids,contratista_name,valores,detalles,...,circunscripcion.electoral,organizacion.politica,identificacion.candidato,nombre.candidato,elegido,valor,tipo.donacion,concepto.del.ingreso,identificacion_aportante,aportante.original.nombre
1,1,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
2,2,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
8,8,31997,20-12-10967566,2020-07-29T17:21:28.000Z,NaN,ATLANTICO - INSTITUCION EDUCATIVA DISTRITAL CO...,890900943,COLOMBIANA DE COMERCIO S.A,13941000.0,"Suministro De Nueve (9) Computadores Marca HP,...",...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
9,9,31997,20-12-10967566,2020-07-29T17:21:28.000Z,NaN,ATLANTICO - INSTITUCION EDUCATIVA DISTRITAL CO...,890900943,COLOMBIANA DE COMERCIO S.A,13941000.0,"Suministro De Nueve (9) Computadores Marca HP,...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
15,15,53728,20-4-10746585,2020-05-09T10:07:42.000Z,SANTA ROSA DEL SUR,CALDAS - INSTITUCION EDUCATIVA GOMEZ FERNANDEZ...,890900943,Colombiana de Comercio S.A,5119000.0,Suministro de Impresora y Resmas de papel con ...,...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,899,82495,20-4-10871561,2020-06-26T13:36:12.000Z,CALI,VALLE DEL CAUCA - SOCIEDAD DE ACUEDUCTOS Y ALC...,890301753,SI S.A.S,2244935.0,SUMINISTRO DE ELEMENTOS DE BIOSEGURIDAD (TOALL...,...,Departamental y Distrito Capital,PARTIDO CENTRO DEMOCRATICO,94503511,CHRISTIAN MUNIR GARCES ALJURE,Si,"$5,000,000",Contribución,CONTRIBUCION,890301753,ALMACENES SI S.A.
901,901,87665,20-4-10634988,2020-03-30T17:06:31.000Z,NaN,ANTIOQUIA - CORPORACION RUTA N MEDELLIN,890980724,FUNDACIoN BANCOLOMBIA,0.0,"Aunar esfuerzos técnicos, administrativos y fi...",...,Municipal,CENTRO DEMOCRATICO,70563378,JUAN CARLOS ROBERTO VELEZ URIBE,No,"$20,000",Donación,DONACION,890980724,FUNDACION BANCOLOMBIA
902,902,87665,20-4-10634988,2020-03-30T17:06:31.000Z,NaN,ANTIOQUIA - CORPORACION RUTA N MEDELLIN,890980724,FUNDACIoN BANCOLOMBIA,0.0,"Aunar esfuerzos técnicos, administrativos y fi...",...,Municipal,CENTRO DEMOCRATICO,70563378,JUAN CARLOS ROBERTO VELEZ URIBE,No,"$19,980,000",Donación,DONACION,890980724,FUNDACION BANCOLOMBIA
907,907,87665,20-4-10634988,2020-03-30T17:06:31.000Z,NaN,ANTIOQUIA - CORPORACION RUTA N MEDELLIN,890980724,FUNDACIoN BANCOLOMBIA,0.0,"Aunar esfuerzos técnicos, administrativos y fi...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$20,000,000",Donación,DONACIÓN,890980724,FUNDACION BANCOLOMBIA


In [149]:
df_financiacion = financiacion.copy()



# Formatting with $ 
df_financiacion['valores'] = df_financiacion['valores'].map('${:,.0f}'.format)
# Formatting with capitalization
df_financiacion['detalles'] = df_financiacion['detalles'].str.capitalize()
df_financiacion['contratista_name'] = df_financiacion['contratista_name'].str.title()
df_financiacion['departamento_def'] = df_financiacion['departamento_def'].str.title()
df_financiacion['municipio_contrato'] = df_financiacion['municipio_contrato'].str.title()
df_financiacion['organizacion.politica'] = df_financiacion['organizacion.politica'].str.title()
df_financiacion['nombre.candidato'] = df_financiacion['nombre.candidato'].str.title()

# Formatting nulls of municipio
df_financiacion['municipio_contrato'] = df_financiacion['municipio_contrato'].fillna('')
# Formatting fechas with only date
df_financiacion['date'] = pd.to_datetime(df_financiacion['fechas']).dt.date
# Getting dates' month
df_financiacion['month'] =pd.to_datetime(df_financiacion['fechas']).dt.to_period("M")



df_financiacion = df_financiacion.rename(columns={
    'entidades': 'Nombre de la entidad',
    'departamento_def': 'Departamento contrato',
    'municipio_contrato': 'Municipio',
    'date': 'Fecha del contrato',
    'detalles': 'Descripcion del contrato',
    'contratista_name': 'Nombre Financiador',
    'contratista_ids': 'ID Financiador',
    'uri': 'SECOP URL',
    'valores': 'Valor del contrato',
    'valor': 'Valor del aporte',
    'month': 'Mes del contrato',
    'elegido': 'Candidato elegido',
    'organizacion.politica': 'Organización Política',
    'nombre.candidato': 'Nombre Candidato'
})


df_financiacion_final = df_financiacion[['Fecha del contrato', 'Departamento contrato',
                                   'Nombre de la entidad', 'Nombre Financiador', 
                                    'ID Financiador', 'Valor del contrato',
                                     'Descripcion del contrato', 'Valor del aporte',
                                 'Organización Política','Nombre Candidato', 'SECOP URL' ]]
df_financiacion_final = df_financiacion_final.sort_values(by='Fecha del contrato', ascending=False)
df_financiacion_final.head()

/Users/lmontesmarti/Documents/GitHub/canopy-demo/venv/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1091: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,Fecha del contrato,Departamento contrato,Nombre de la entidad,Nombre Financiador,ID Financiador,Valor del contrato,Descripcion del contrato,Valor del aporte,Organización Política,Nombre Candidato,SECOP URL
747,2020-09-30,Valle Del Cauca,VALLE DEL CAUCA - ESE HOSPITAL NUESTRA SEÑORA ...,Distracom S.A,811009788,"$5,000,000",El contratista se obliga con el contratante a ...,"$5,000,000",Partido Centro Democratico,Juan Fernando Espinal Ramirez,https://www.contratos.gov.co/consultas/detalle...
749,2020-09-30,Valle Del Cauca,VALLE DEL CAUCA - ESE HOSPITAL NUESTRA SEÑORA ...,Distracom S.A,811009788,"$5,000,000",El contratista se obliga con el contratante a ...,"$50,000,000",Partido Social De Unidad Nacional Partido De La U,German Dario Hoyos Giraldo,https://www.contratos.gov.co/consultas/detalle...
96,2020-09-30,Bogotá,BOGOTÁ D.C. - ALCALDÍA LOCAL LOS MARTIRES,Caja De Compensacion Familiar - Compensar,860066942,"$773,946,197",Compensar se obliga a prestar los servicios re...,"$222,621",Bogota Adelante,Rafael Pardo Rueda,https://www.contratos.gov.co/consultas/detalle...
748,2020-09-30,Valle Del Cauca,VALLE DEL CAUCA - ESE HOSPITAL NUESTRA SEÑORA ...,Distracom S.A,811009788,"$5,000,000",El contratista se obliga con el contratante a ...,"$10,000,000",Partido Centro Democratico,Paola Andrea Holguin Moreno,https://www.contratos.gov.co/consultas/detalle...
113,2020-09-29,NaN,ATLANTICO - INSTITUCION EDUCATIVA DISTRITAL KA...,Yesenia Montero Leybas,32844355,"$7,000,000",Suministro de impresión y entrega de fotocopia...,"$1,000,000",Partido Social De Unidad Nacional Partido De La U,Juan Ospino Acuña,https://www.contratos.gov.co/consultas/detalle...


In [150]:
df_financiacion_final['Departamento contrato'].fillna('Sin Definir').drop_duplicates()

747       Valle Del Cauca
96                 Bogotá
113           Sin Definir
125             Santander
839                Tolima
693               Vichada
236             Antioquia
679               Bolívar
875    Norte De Santander
51                   Meta
741             Risaralda
719                 Cesar
694               Córdoba
704               Caquetá
626                 Sucre
833                Nariño
667          Cundinamarca
834             Atlántico
746                 Cauca
785                Boyacá
881              Putumayo
578                Caldas
700                 Huila
695              Casanare
709                 Chocó
835                Vaupés
711             Magdalena
829               Quindio
610              Amazonas
722            La Guajira
623              Medellín
Name: Departamento contrato, dtype: object

In [151]:
graph_f_df = df_financiacion.groupby(['Organización Política']).size().reset_index(name='Num contratos')
graph_f_df = graph_f_df.sort_values(by='Num contratos')
graph_f_df = graph_f_df.iloc[-11:-1,:]

fig = px.bar(graph_f_df, x="Num contratos", y="Organización Política", orientation='h',
             height=500)
fig.update_layout(
    font=dict(
        color="#252525",
        family="Roboto"
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_title="Numero de contratos asignados a financiadores",
    yaxis_title='',
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.show()

# 3. Concentración contratistas

In [7]:
concentracion = pd.read_csv('s3://proyectocanopy/alarma_concentracion_valor_contratistas.csv')
concentracion = concentracion.drop_duplicates(subset=['contratista_ids', 'departamento_def'])
df_national_raw = concentracion.copy()
concentracion

,Unnamed: 0,departamento_def,contratista_ids,val_contratista,val_depto,pct_def,contratista_name
0,0,amazonas,7634220,19000000.0,2.301949e+09,0.825388,HAROLDO RAFAEL ODUVER GoMEZ
1,1,amazonas,15877822,419815500.0,2.301949e+09,18.237393,SEGUNDO HIPOLITO ARCENTALES VARGAS
2,2,amazonas,17625757,25200000.0,2.301949e+09,1.094724,VIDRIOS MOTA
3,3,amazonas,100292219,10112000.0,2.301949e+09,0.439280,KAREN ROSSANA GUTIERREZ VIVEROS
4,4,amazonas,112120308,6300000.0,2.301949e+09,0.273681,MARBIN DARIO PAREDES
...,...,...,...,...,...,...,...
20992,20992,vichada,86074661,51675000.0,1.142489e+09,4.523020,JORGE ENRIQUE CALDERON RINCON
20993,20993,vichada,900129462,19900000.0,1.142489e+09,1.741811,COMPAÑiA DE PREVISIoN EXEQUIAL NACIONAL DE SER...
20995,20995,vichada,900351693,10408983.0,1.142489e+09,0.911080,THE SAFETY SHOP S.A.S
20996,20996,vichada,900484765,15000000.0,1.142489e+09,1.312923,FUNDACIoN EL ROCIO DE LA MAÑANA


In [213]:
concentracion['RN'] = concentracion.sort_values(['pct_def'], ascending=[False]) \
             .groupby(['departamento_def']) \
             .cumcount() + 1
concentracion = concentracion[concentracion['RN']<=10]

In [214]:
df_proov = concentracion

# Formatting with $ 
df_proov['val_contratista'] = df_proov['val_contratista'].map('${:,.0f}'.format)
# Formatting with %
df_proov_2 = df_proov.copy()
df_proov['pct_def'] = df_proov['pct_def'].map('{:.2f}%'.format)
# Formatting with capitalization
df_proov['departamento_def'] = df_proov['departamento_def'].str.capitalize()
df_proov['contratista_name'] = df_proov['contratista_name'].str.title()

df_proov = df_proov.rename(columns={'contratista_ids':'Documento proveedor',
    'departamento_def': 'Departamento',
    'val_contratista': 'Valor de contratos',
    'pct_def': 'Pct de contratos',
    'contratista_name': 'Proveedor adjudicado'})

df_proov = df_proov[['Departamento', 'Proveedor adjudicado', 'Documento proveedor', 'Valor de contratos', 'Pct de contratos']]
df_proov = df_proov.sort_values(by='Pct de contratos', ascending=False)
df_proov

/Users/lmontesmarti/Documents/GitHub/canopy-demo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/lmontesmarti/Documents/GitHub/canopy-demo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/lmontesmarti/Documents/GitHub/canopy-demo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,Departamento,Proveedor adjudicado,Documento proveedor,Valor de contratos,Pct de contratos
11944,Guainía,Clinica Salud Y Vida S.A.S.,901203586,"$56,410,000",9.77%
9529,Cesar,Consorcio Quedate En Casa,901380613,"$1,827,956,505",9.75%
58,Amazonas,Erika Paola Sosa Llontop,444927,"$220,117,000",9.56%
16770,Quindio,Guitarra And Coffee S.A.S,901083690,"$300,000,000",9.54%
20899,Vaupés,Panamericana De Quimicos,40404298,"$177,647,400",9.53%
...,...,...,...,...,...
3768,Bogotá,Colegio Mayor De Nuestra Señora Del Rosario,8600077593,"$1,876,051,758",0.51%
3670,Bogotá,Sodexo Servicios De Beneficios E Incentivos Co...,800219876,"$1,846,271,000",0.50%
3773,Bogotá,Universidad De La Salle,860015542,"$1,734,951,835",0.47%
3782,Bogotá,Caja De Compensacion Familiar Compensar,86006694,"$1,617,214,467",0.44%


In [215]:
import numpy as np
# Códigos para el mapa
df_codes = pd.read_csv("https://raw.githubusercontent.com/melissamnt/code_utils/master/csv_department_codes.csv",
                   dtype={"cod": str})
df_codes['Code'] = df_codes['Code'].astype('str')
df_codes = df_codes.replace({'5': '05', '8': '08'})

#Concentracion acumulada
concentracion_depto = df_proov_2.groupby(['departamento_def'])['pct_def'].sum().reset_index(name='pct_def')
concentracion_depto.head(2)

,departamento_def,pct_def
0,amazonas,66.536115
1,antioquia,25.824060


In [220]:
# 4. Dataframe SECOP I & II concentracion top 10 contratistas por depto

df_proov_cum = concentracion_depto.rename(columns={'departamento_def':'departamento', 'pct_def': 'Pct acumulado proveedores'})

df_proov_cum = df_proov_cum.sort_values(by='Pct acumulado proveedores', ascending=False)
# Codes for map
df_proov_cum['Departamento_upper'] = df_proov_cum['departamento'].str.upper()
df_proov_cum['Departamento_upper'] = df_proov_cum['Departamento_upper'].replace({'QUINDIO': 'QUINDÍO'})
df_proov_cum = pd.merge(df_proov_cum, df_codes, left_on='Departamento_upper',  right_on='Departamento', how='left')
df_proov_cum = df_proov_cum.drop(['Departamento_upper', 'Departamento'], axis=1)
# Formatting
df_proov_cum['departamento'] = df_proov_cum['departamento'].str.title() 
df_proov_cum['Pct acumulado proveedores'] = np.round(df_proov_cum['Pct acumulado proveedores'], 2)
df_proov_cum = df_proov_cum.rename(columns={'departamento':'Departamento'})

df_proov_cum.loc[df_proov_cum['Departamento']=='San Andrés, Providencia y Santa Catalina', 'Departamento'] = 'San Andrés'

In [221]:
df_proov_cum

,Departamento,Pct acumulado proveedores,Code
0,Atlántico,72.81,08
1,Guainía,72.62,94
2,Quindio,72.45,63
3,Guaviare,70.85,95
4,Tolima,70.66,73
5,Amazonas,66.54,91
6,Bolívar,66.03,13
7,La Guajira,63.81,44
8,Vaupés,63.39,97
9,Vichada,55.38,99


In [228]:
# 5. Dataframe SECOP I & II nombre de top 10 contratistas por depto
df_proov = pd.read_csv('s3://proyectocanopy/alarma_concentracion_valor_contratistas.csv')
df_proov = df_proov.drop_duplicates(subset=['contratista_ids', 'departamento_def'])

# Only first 10 
df_proov['RN'] = df_proov.sort_values(['pct_def'], ascending=[False]) \
             .groupby(['departamento_def']) \
             .cumcount() + 1
df_proov = df_proov[df_proov['RN']<=10]

# Formatting with $ 
df_proov['val_contratista'] = df_proov['val_contratista'].map('${:,.0f}'.format)
# Formatting with %
df_proov_2 = df_proov.copy()
df_proov['pct_def'] = df_proov['pct_def'].map('{:.2f}'.format)
# Formatting with capitalization
df_proov['departamento_def'] = df_proov['departamento_def'].str.capitalize()
df_proov['contratista_name'] = df_proov['contratista_name'].str.title()

df_proov = df_proov.rename(columns={'contratista_ids':'Documento proveedor',
    'departamento_def': 'Departamento',
    'val_contratista': 'Valor de contratos',
    'pct_def': 'Pct de contratos',
    'contratista_name': 'Proveedor adjudicado'})

df_proov = df_proov[['Departamento', 'Proveedor adjudicado', 'Documento proveedor', 'Valor de contratos', 'Pct de contratos']]
df_proov = df_proov.sort_values(by='Pct de contratos', ascending=False)


# 4. Dataframe SECOP I & II concentracion top 10 contratistas por depto
df_proov_cum = df_proov_2.groupby(['departamento_def'])['pct_def'].sum().reset_index(name='pct_def')
df_proov_cum = df_proov_cum.rename(columns={'departamento_def':'departamento', 'pct_def': 'Pct acumulado proveedores'})

# Codes for map
df_proov_cum['Departamento_upper'] = df_proov_cum['departamento'].str.upper()
df_proov_cum['Departamento_upper'] = df_proov_cum['Departamento_upper'].replace({'QUINDIO': 'QUINDÍO'})
df_proov_cum = pd.merge(df_proov_cum, df_codes, left_on='Departamento_upper',  right_on='Departamento', how='left')
df_proov_cum = df_proov_cum.drop(['Departamento_upper', 'Departamento'], axis=1)
# Formatting
df_proov_cum['departamento'] = df_proov_cum['departamento'].str.title() 
df_proov_cum['Pct acumulado proveedores'] = np.round(df_proov_cum['Pct acumulado proveedores'], 2)
df_proov_cum = df_proov_cum.rename(columns={'departamento':'Departamento'})
df_proov_cum = df_proov_cum.sort_values(by='Pct acumulado proveedores', ascending=False)

df_proov_cum.loc[df_proov_cum['Departamento']=='San Andrés, Providencia y Santa Catalina', 'Departamento'] = 'San Andrés'

In [229]:
df_proov_cum

,Departamento,Pct acumulado proveedores,Code
3,Atlántico,72.81,08
15,Guainía,72.62,94
24,Quindio,72.45,63
16,Guaviare,70.85,95
28,Tolima,70.66,73
0,Amazonas,66.54,91
5,Bolívar,66.03,13
18,La Guajira,63.81,44
30,Vaupés,63.39,97
31,Vichada,55.38,99


In [2]:
import pandas as pd
pd.DataFrame.from_dict({'1':['A', 'B']}, '2': )

,1
0,A
1,B


## Mapa global

In [8]:
# 3. Dataframe SECOP I & II map (number of COVID contracts)
df_national_raw.head()

,Unnamed: 0,departamento_def,contratista_ids,val_contratista,val_depto,pct_def,contratista_name
0,0,amazonas,7634220,19000000.0,2.301949e+09,0.825388,HAROLDO RAFAEL ODUVER GoMEZ
1,1,amazonas,15877822,419815500.0,2.301949e+09,18.237393,SEGUNDO HIPOLITO ARCENTALES VARGAS
2,2,amazonas,17625757,25200000.0,2.301949e+09,1.094724,VIDRIOS MOTA
3,3,amazonas,100292219,10112000.0,2.301949e+09,0.439280,KAREN ROSSANA GUTIERREZ VIVEROS
4,4,amazonas,112120308,6300000.0,2.301949e+09,0.273681,MARBIN DARIO PAREDES


In [24]:
# Grouping by number and sum of contracts
df_national_covid = df_national_raw.groupby(['departamento_def']).agg(
    num_contratos=('val_contratista', 'count'),
    sum_contratos=('val_contratista', 'sum'),
).reset_index()

# Formatting with capitalization
df_national_covid['departamento_def'] = df_national_covid['departamento_def'].str.capitalize()
# Upper for joins 
df_national_covid['Departamento_upper'] = df_national_covid['departamento_def'].str.upper()
df_national_covid['Departamento_upper'] = df_national_covid['Departamento_upper'].replace('QUINDIO', 'QUINDÍO')

# Joining with codes
df_codes = pd.read_csv("https://raw.githubusercontent.com/melissamnt/code_utils/master/csv_department_codes.csv",
                   dtype={"cod": str})
df_codes['Code'] = df_codes['Code'].astype('str')
df_codes = df_codes.replace({'5': '05', '8': '08'})
df_national_covid = pd.merge(df_national_covid, df_codes, left_on='Departamento_upper',  right_on='Departamento', how='left')

# Rename colums
df_national_covid = df_national_covid.drop('Departamento', axis=1)
df_national_covid = df_national_covid.rename(columns={'departamento_def': 'Departamento',
                                                     'num_contratos': 'Num. contratos',
                                                     'sum_contratos': 'Total cuantía contratos'})
df_national_covid = df_national_covid[['Departamento', 'Num. contratos', 'Total cuantía contratos', 'Code']]
df_national_covid

,Departamento,Num. contratos,Total cuantía contratos,Code
0,Amazonas,56,2.201961e+09,91
1,Antioquia,2005,8.191774e+10,05
2,Arauca,181,3.445538e+09,81
3,Atlántico,368,4.182361e+10,08
4,Bogotá,178,1.494153e+11,NaN
5,Bolívar,953,1.042192e+11,13
6,Boyacá,713,2.702250e+10,15
7,Caldas,570,1.037361e+10,17
8,Caquetá,156,5.141448e+09,18
9,Casanare,152,5.633885e+09,85
